# OpenAI Function Calling In LangChain

In [4]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [5]:
from typing import List
from pydantic import BaseModel, Field

## Pydantic Syntax

Pydantic data classes are a blend of Python's data classes with the validation power of Pydantic. 

They offer a concise way to define data structures while ensuring that the data adheres to specified types and constraints.

In standard python you would create a class like this:

In [3]:
class User:
    def __init__(self, name: str, age: int, email: str):
        self.name = name
        self.age = age
        self.email = email

In [4]:
foo = User(name="Joe",age=32, email="joe@gmail.com")

In [5]:
foo.name

'Joe'

In [6]:
# 故意輸入錯誤
foo = User(name="Joe",age="bar", email="joe@gmail.com")

In [7]:
# 輸出錯誤資訊
foo.age

'bar'

In [8]:
# 改用 pydantic 的 BaseModel
class pUser(BaseModel):
    name: str
    age: int
    email: str

In [9]:
foo_p = pUser(name="Jane", age=32, email="jane@gmail.com")

In [10]:
foo_p.name

'Jane'

<p style=\"background-color:#F5C780; padding:15px\"><b>Note:</b> The next line is expected to fail.</p>

In [11]:
foo_p = pUser(name="Jane", age="bar", email="jane@gmail.com")

ValidationError: 1 validation error for pUser
age
  value is not a valid integer (type=type_error.integer)

In [12]:
class Class(BaseModel):
    students: List[pUser]

In [13]:
obj = Class(
    students=[pUser(name="Jane", age=32, email="jane@gmail.com")]
)

In [14]:
obj

Class(students=[pUser(name='Jane', age=32, email='jane@gmail.com')])

## Pydantic to OpenAI function definition


In [15]:
class WeatherSearch(BaseModel):
    # 有 Docstring
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str = Field(description="airport code to get weather for")

In [16]:
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

In [17]:
weather_function = convert_pydantic_to_openai_function(WeatherSearch)

In [18]:
weather_function

{'name': 'WeatherSearch',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'title': 'WeatherSearch',
  'description': 'Call this with an airport code to get the weather at that airport',
  'type': 'object',
  'properties': {'airport_code': {'title': 'Airport Code',
    'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code']}}

In [23]:
class WeatherSearch1(BaseModel):
    # 沒有 Docstring
    airport_code: str = Field(description="airport code to get weather for")

<p style=\"background-color:#F5C780; padding:15px\"><b>Note:</b> The next cell is expected to generate an error.</p>

In [24]:
# 舊版的 LangChain 才會錯
# 新版的就不會
convert_pydantic_to_openai_function(WeatherSearch1)

KeyError: 'description'

In [1]:
# 安裝新版 langChain

# %pip install -U langchain langchain-core langchain-community langchain-openai

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: pip3 install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_core.utils.function_calling import convert_to_openai_function

In [6]:
class WeatherSearch1(BaseModel):
    # 沒有 Docstring
    airport_code: str = Field(description="airport code to get weather for")

In [8]:
# 新版不用 Docstring 也能順利執行
convert_to_openai_function(WeatherSearch1)

{'name': 'WeatherSearch1',
 'description': '',
 'parameters': {'properties': {'airport_code': {'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [25]:
class WeatherSearch2(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str

In [26]:
convert_pydantic_to_openai_function(WeatherSearch2)

{'name': 'WeatherSearch2',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'title': 'WeatherSearch2',
  'description': 'Call this with an airport code to get the weather at that airport',
  'type': 'object',
  'properties': {'airport_code': {'title': 'Airport Code', 'type': 'string'}},
  'required': ['airport_code']}}

In [27]:
from langchain.chat_models import ChatOpenAI

In [28]:
model = ChatOpenAI()

In [29]:
model.invoke("what is the weather in SF today?", functions=[weather_function])

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{"airport_code":"SFO"}'}})

In [30]:
model_with_function = model.bind(functions=[weather_function])

In [31]:
model_with_function.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{"airport_code":"SFO"}'}})

## Forcing it to use a function

We can force the model to use a function

In [32]:
model_with_forced_function = model.bind(functions=[weather_function], function_call={"name":"WeatherSearch"})

In [33]:
model_with_forced_function.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{"airport_code":"SFO"}'}})

In [34]:
model_with_forced_function.invoke("hi!")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{"airport_code":"SFO"}'}})

## Using in a chain

We can use this model bound to function in a chain as we normally would

In [35]:
from langchain.prompts import ChatPromptTemplate

In [36]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("user", "{input}")
])

In [37]:
chain = prompt | model_with_function

In [38]:
chain.invoke({"input": "what is the weather in sf?"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{"airport_code":"SFO"}'}})

## Using multiple functions

Even better, we can pass a set of function and let the LLM decide which to use based on the question context.

In [39]:
class ArtistSearch(BaseModel):
    """Call this to get the names of songs by a particular artist"""
    artist_name: str = Field(description="name of artist to look up")
    n: int = Field(description="number of results")

In [40]:
functions = [
    convert_pydantic_to_openai_function(WeatherSearch),
    convert_pydantic_to_openai_function(ArtistSearch),
]

In [41]:
model_with_functions = model.bind(functions=functions)

In [42]:
model_with_functions.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{"airport_code":"SFO"}'}})

In [43]:
model_with_functions.invoke("what are three songs by taylor swift?")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'ArtistSearch', 'arguments': '{"artist_name":"Taylor Swift","n":3}'}})

In [44]:
model_with_functions.invoke("hi!")

AIMessage(content='Hello! How can I assist you today?')